In [1]:
import pandas as pd
from random import randint

def generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, 
                  dataset_size = 100000):
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d
  


d = generate_data(dataset_size = 100000)
d



,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,539,294,10,3757,646,7,8,2008,54,5
1,1093,221,2,19562,619,5,45,2002,7,5
2,2773,361,9,10034,321,3,48,2015,151,1
3,1519,238,5,15230,366,4,7,2020,59,5
4,2498,57,10,8437,197,8,39,2018,157,7
...,...,...,...,...,...,...,...,...,...,...
99995,2562,119,3,10592,200,5,36,2003,194,3
99996,2484,341,6,18981,201,9,43,2015,181,7
99997,1844,125,5,21359,501,1,30,2014,64,3
99998,1150,67,3,1980,383,10,31,2021,61,3


In [2]:
d.shape

(100000, 10)

In [3]:

d.to_csv('data.csv', index = False)

d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   book_id       100000 non-null  int64
 1   author_id     100000 non-null  int64
 2   book_genre    100000 non-null  int64
 3   reader_id     100000 non-null  int64
 4   num_pages     100000 non-null  int64
 5   book_rating   100000 non-null  int64
 6   publisher_id  100000 non-null  int64
 7   publish_year  100000 non-null  int64
 8   book_price    100000 non-null  int64
 9   text_lang     100000 non-null  int64
dtypes: int64(10)
memory usage: 8.4 MB


In [4]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

def normalize(pred_ratings):
    '''
    This function will normalize the input pred_ratings
    
    params:
        pred_ratings (List -> List) : The prediction ratings 
    '''
    return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())
  
def generate_prediction_df(mat, pt_df, n_factors):
    '''
    This function will calculate the single value decomposition of the input matrix
    given n_factors. It will then generate and normalize the user rating predictions.
    
    params:
        mat (CSR Matrix) : scipy csr matrix corresponding to the pivot table (pt_df)
        pt_df (DataFrame) : pandas dataframe which is a pivot table
        n_factors (Integer) : Number of singular values and vectors to compute. 
                              Must be 1 <= n_factors < min(mat.shape). 
    '''
    
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
        
    # matrix factorization
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # calculate pred ratings
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize(pred_ratings)
    
    # convert to df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df

def recommend_items(pred_df, usr_id, n_recs):
    '''
    Given a usr_id and pred_df this function will recommend
    items to the user.
    
    params:
        pred_df (DataFrame) : generated from `generate_prediction_df` function
        usr_id (Integer) : The user you wish to get item recommendations for
        n_recs (Integer) : The number of recommendations you want for this user
    '''
    
    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    return rec_df
  
if __name__ == '__main__':

    # import data
    df = pd.read_csv('data.csv')
#     print(df.shape)

    # generate a pivot table with readers on the index and books on the column and values being the ratings
    pt_df = df.pivot_table(
        columns = 'book_id',
        index = 'reader_id',
        values = 'book_rating'
    ).fillna(0)

    # convert to a csr matrix
    mat = pt_df.values
    mat = csr_matrix(mat)
    
    pred_df = generate_prediction_df(mat, pt_df, 10)

    # generate recommendations
    print(recommend_items(pred_df, 5, 5))

   book_id       sim
0     1719  0.167925
1     2595  0.155720
2     2130  0.152771
3     2086  0.152770
4     1434  0.152171


In [8]:
mat.shape

(28907, 3000)

In [10]:
pred_df

,1,2,3,4,5,6,8,9,10,11,...,29990,29991,29992,29993,29995,29996,29997,29998,29999,30000
book_id,,,,,,,,,,,,,,,,,,,,,
1,0.145383,0.147115,0.147917,0.147444,0.147403,0.147033,0.147717,0.150724,0.148741,0.148266,...,0.147256,0.147276,0.151178,0.146587,0.147644,0.147262,0.147263,0.147345,0.148577,0.148015
2,0.146911,0.147357,0.147282,0.147404,0.147808,0.147923,0.147412,0.147522,0.147991,0.147148,...,0.147045,0.147894,0.148389,0.147277,0.147416,0.147332,0.147848,0.148409,0.148328,0.147885
3,0.146736,0.147210,0.147572,0.148059,0.147235,0.148114,0.148249,0.146921,0.147745,0.146460,...,0.147458,0.148243,0.146089,0.148046,0.147174,0.147289,0.147420,0.148817,0.148386,0.147739
4,0.146488,0.147142,0.148007,0.146906,0.147117,0.147073,0.147654,0.150110,0.148118,0.149861,...,0.146755,0.146938,0.152178,0.147176,0.147556,0.147347,0.147301,0.147088,0.147027,0.147867
5,0.145829,0.147403,0.147567,0.147435,0.148962,0.146853,0.148131,0.148688,0.148696,0.147457,...,0.147053,0.147754,0.147418,0.146464,0.147169,0.147228,0.147839,0.149346,0.148421,0.147309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.147122,0.146915,0.148703,0.146943,0.146829,0.146071,0.148258,0.153279,0.148400,0.148081,...,0.147632,0.146709,0.158280,0.147578,0.148471,0.147325,0.147037,0.147437,0.147646,0.149729
2997,0.146065,0.147225,0.147735,0.147588,0.147062,0.149563,0.148962,0.147790,0.148736,0.147598,...,0.149639,0.148091,0.148095,0.147740,0.147210,0.147648,0.147225,0.147675,0.147873,0.148340
2998,0.147752,0.147256,0.147342,0.147363,0.147149,0.148460,0.147801,0.147157,0.147681,0.148082,...,0.148252,0.147716,0.147998,0.147934,0.147311,0.147422,0.147286,0.147604,0.147576,0.147588


In [2]:
import pandas as pd 
df = pd.read_csv('/home/echelon/Downloads/6. India Census 2011.csv') 
df

,District_code,State_name,District_name,Population,Male,Female,Literate,Workers,Male_Workers,Female_Workers,...,Christians,Sikhs,Buddhists,Jains,Secondary_Education,Higher_Education,Graduate_Education,Age_Group_0_29,Age_Group_30_49,Age_Group_50
0,1,JAMMU AND KASHMIR,Kupwara,870354,474190,396164,439654,229064,190899,38165,...,1700,5600,66,39,74948,39709,21751,600759,178435,89679
1,2,JAMMU AND KASHMIR,Badgam,753745,398041,355704,335649,214866,162578,52288,...,1489,5559,47,6,66459,41367,27950,503223,160933,88978
2,3,JAMMU AND KASHMIR,Leh(Ladakh),133487,78971,54516,93770,75079,53265,21814,...,658,1092,88635,103,16265,8923,6197,70703,41515,21019
3,4,JAMMU AND KASHMIR,Kargil,140802,77785,63017,86236,51873,39839,12034,...,604,1171,20126,28,16938,9826,3077,87532,35561,17488
4,5,JAMMU AND KASHMIR,Punch,476835,251899,224936,261724,161393,117677,43716,...,958,11188,83,10,46062,29517,13962,304979,109818,61334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,636,PONDICHERRY,Mahe,41816,19143,22673,36470,11802,9093,2709,...,958,3,21,14,8773,5711,4475,19760,12543,9428
636,637,PONDICHERRY,Karaikal,200222,97809,102413,154916,68301,53139,15162,...,18391,46,63,54,29456,15930,18998,102333,60089,37434
637,638,ANDAMAN AND NICOBAR ISLANDS,Nicobars,36842,20727,16115,25332,17125,12714,4411,...,26077,343,119,0,6325,3668,1300,18786,12088,5922
638,639,ANDAMAN AND NICOBAR ISLANDS,North AND Middle Andaman,105597,54861,50736,78683,38579,30612,7967,...,21870,77,84,4,12707,10042,5133,57350,30741,17464


In [3]:
df.shape

(640, 25)

In [4]:
df.columns

Index(['District_code', 'State_name', 'District_name', 'Population', 'Male',
       'Female', 'Literate', 'Workers', 'Male_Workers', 'Female_Workers',
       'Cultivator_Workers', 'Agricultural_Workers', 'Household_Workers',
       'Hindus', 'Muslims', 'Christians', 'Sikhs', 'Buddhists', 'Jains',
       'Secondary_Education', 'Higher_Education', 'Graduate_Education',
       'Age_Group_0_29', 'Age_Group_30_49', 'Age_Group_50'],
      dtype='object')

In [5]:
df[['Age_Group_0_29','Muslims','State_name','Population','Male','Female','Hindus']]

,Age_Group_0_29,Muslims,State_name,Population,Male,Female,Hindus
0,600759,823286,JAMMU AND KASHMIR,870354,474190,396164,37128
1,503223,736054,JAMMU AND KASHMIR,753745,398041,355704,10110
2,70703,19057,JAMMU AND KASHMIR,133487,78971,54516,22882
3,87532,108239,JAMMU AND KASHMIR,140802,77785,63017,10341
4,304979,431279,JAMMU AND KASHMIR,476835,251899,224936,32604
...,...,...,...,...,...,...,...
635,19760,12856,PONDICHERRY,41816,19143,22673,27940
636,102333,28835,PONDICHERRY,200222,97809,102413,152638
637,18786,1336,ANDAMAN AND NICOBAR ISLANDS,36842,20727,16115,8681
638,57350,2165,ANDAMAN AND NICOBAR ISLANDS,105597,54861,50736,81163


In [5]:
import pandas as pd 
csv_df = pd.read_csv('/home/echelon/Music/Join_data_not_clean.csv') 
csv_df

,MOBILE_NUMBER,CIRCLE_ID,CIRCLE_NAME,LOC_OG_T2O_MOU,STD_OG_T2O_MOU,LOC_IC_T2O_MOU,LAST_DATE_OF_MONTH,ARPU,ONNET_MOU,OFFNET_MOU,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlyrecharges,totalcharges,churn
0,7000000256,305,BH,0.0,0.0,0.0,31-07-2014 00:00,1260.65,428.23,1488.51,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
1,7000000256,305,BH,0.0,0.0,0.0,31-08-2014 00:00,1484.67,784.58,1557.94,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
2,7000000256,305,BH,0.0,0.0,0.0,30-09-2014 00:00,1128.19,2476.36,427.86,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
3,7000000267,302,KOL,0.0,0.0,0.0,30-06-2014 00:00,405.76,54.88,93.68,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
4,7000000267,302,KOL,0.0,0.0,0.0,31-07-2014 00:00,232.14,100.76,118.66,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398290,7002402996,304,ORI,0.0,0.0,0.0,31-08-2014 00:00,722.37,1074.98,304.48,...,No,No,No,No,Month-to-month,Yes,Electronic check,76.10,1712.70,Yes
398291,7002402996,304,ORI,0.0,0.0,0.0,30-09-2014 00:00,734.42,1308.29,302.79,...,Yes,No,No,No,One year,No,Mailed check,61.20,2021.20,No
398292,7002403039,302,KOL,0.0,0.0,0.0,30-06-2014 00:00,142.14,0.31,18.33,...,Yes,Yes,No,No,One year,No,Mailed check,86.80,1940.80,No
398293,7002403039,302,KOL,0.0,0.0,0.0,31-07-2014 00:00,170.91,6.46,27.43,...,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,89.35,567.80,No


In [6]:
csv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398295 entries, 0 to 398294
Data columns (total 86 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   MOBILE_NUMBER           398295 non-null  int64  
 1   CIRCLE_ID               398295 non-null  int64  
 2   CIRCLE_NAME             398295 non-null  object 
 3   LOC_OG_T2O_MOU          394895 non-null  float64
 4   STD_OG_T2O_MOU          394895 non-null  float64
 5   LOC_IC_T2O_MOU          394895 non-null  float64
 6   LAST_DATE_OF_MONTH      396636 non-null  object 
 7   ARPU                    398295 non-null  float64
 8   ONNET_MOU               379077 non-null  float64
 9   OFFNET_MOU              379077 non-null  float64
 10  ROAM_IC_MOU             379077 non-null  float64
 11  LOC_OG_T2T_MO           379077 non-null  float64
 12  LOC_OG_T2M_MOU          379077 non-null  float64
 13  LOC_OG_T2F_MOU          379077 non-null  float64
 14  LOC_OG_T2C_MOU      

In [8]:
csv_df.describe()

,MOBILE_NUMBER,CIRCLE_ID,LOC_OG_T2O_MOU,STD_OG_T2O_MOU,LOC_IC_T2O_MOU,ARPU,ONNET_MOU,OFFNET_MOU,ROAM_IC_MOU,LOC_OG_T2T_MO,...,AUG_VBC_3G,JUL_VBC_3G,JUN_VBC_3G,SEP_VBC_3G,LAT,LONGITUDE,seniorcitizen,tenure,monthlyrecharges,totalcharges
count,3.982950e+05,398295.000000,394895.0,394895.0,394895.0,398295.000000,379077.000000,379077.000000,379077.000000,379077.000000,...,398295.000000,398295.000000,398295.000000,398295.000000,398295.000000,398295.000000,398295.000000,398295.000000,398295.000000,398295.000000
mean,7.001208e+09,305.500496,0.0,0.0,0.0,276.758290,132.365034,195.520913,7.699025,46.026407,...,68.401286,67.024359,59.982753,3.313085,24.023161,85.856673,0.169091,35.398898,68.626586,2561.812903
std,6.957357e+05,2.872248,0.0,0.0,0.0,338.662475,305.862801,322.373490,68.383826,151.398505,...,267.759449,271.604905,253.916158,32.476456,2.843272,5.854561,0.374833,23.797641,29.442277,2260.914221
min,7.000000e+09,301.000000,0.0,0.0,0.0,-2258.710000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,18.520210,73.844738,0.000000,2.000000,18.950000,33.600000
25%,7.000606e+09,303.000000,0.0,0.0,0.0,83.560000,6.440000,31.460000,0.000000,1.560000,...,0.000000,0.000000,0.000000,0.000000,22.572645,85.124513,0.000000,12.000000,45.400000,644.350000
50%,7.001205e+09,306.000000,0.0,0.0,0.0,190.910000,32.260000,91.890000,0.000000,11.630000,...,0.000000,0.000000,0.000000,0.000000,23.802123,85.808092,0.000000,33.000000,75.000000,1782.400000
75%,7.001812e+09,308.000000,0.0,0.0,0.0,366.205000,115.640000,226.930000,0.000000,40.040000,...,0.000000,0.000000,0.000000,0.000000,25.759202,91.051914,0.000000,58.000000,91.700000,4131.950000
max,7.002411e+09,310.000000,0.0,0.0,0.0,38805.620000,10752.560000,14007.340000,15371.040000,10752.560000,...,12916.220000,9165.600000,11166.210000,2618.570000,28.634022,93.174326,1.000000,72.000000,116.250000,8468.200000


In [46]:
import pandas as pd 
from pyhive import hive

host='164.52.192.120',
port=10000,
database='default',
username='hive',                               
password='password',                              
auth='CUSTOM'                                                               
# query = 'show partion'
query = 'select * from ml_telco_cdr where circle_name = "ASM"'
# query = 'select distinct circle_name from ml_telco_cdr'
# query = 'select distinct circle_name from ml_telco_cdr'

# '''ml_telco.ml_telco_cdr'''

try:

    conn = hive.connect(host='164.52.192.120',
                    port='10000',
                    database='ml_telco',
                    username='hive',
                    password='password',
                    auth='CUSTOM')
    hive_df = pd.read_sql(query, conn)
#     print(df['ml_telco_cdr.circle_name'].sample(20))
#     print(df.columns)
    print(list(hive_df.columns))
    print('count is running ...')
except Exception as e:
    print(f'{e}')
    
    
# distinct

/home/echelon/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


['ml_telco_cdr.mobile_number', 'ml_telco_cdr.circle_id', 'ml_telco_cdr.circle_name', 'ml_telco_cdr.loc_og_t2o_mou', 'ml_telco_cdr.std_og_t2o_mou', 'ml_telco_cdr.loc_ic_t2o_mou', 'ml_telco_cdr.last_date_of_month', 'ml_telco_cdr.arpu', 'ml_telco_cdr.onnet_mou', 'ml_telco_cdr.offnet_mou', 'ml_telco_cdr.roam_ic_mou', 'ml_telco_cdr.loc_og_t2t_mo', 'ml_telco_cdr.loc_og_t2m_mou', 'ml_telco_cdr.loc_og_t2f_mou', 'ml_telco_cdr.loc_og_t2c_mou', 'ml_telco_cdr.loc_og_mou', 'ml_telco_cdr.std_og_t2t_mou', 'ml_telco_cdr.std_og_t2m_mou', 'ml_telco_cdr.std_og_t2f_mou', 'ml_telco_cdr.std_og_t2c_mou', 'ml_telco_cdr.std_og_mou', 'ml_telco_cdr.isd_og_mou', 'ml_telco_cdr.spl_og_mou', 'ml_telco_cdr.og_others', 'ml_telco_cdr.total_og_mou', 'ml_telco_cdr.loc_ic_t2t_mou', 'ml_telco_cdr.loc_ic_t2m_mou', 'ml_telco_cdr.loc_ic_t2f_mou', 'ml_telco_cdr.loc_ic_mou', 'ml_telco_cdr.std_ic_t2t_mou', 'ml_telco_cdr.std_ic_t2m_mou', 'ml_telco_cdr.std_ic_t2f_mou', 'ml_telco_cdr.std_ic_t2o_mou', 'ml_telco_cdr.std_ic_mou', 'ml_

In [37]:
hive_df

,ml_telco_cdr.mobile_number,ml_telco_cdr.circle_id,ml_telco_cdr.circle_name,ml_telco_cdr.loc_og_t2o_mou,ml_telco_cdr.std_og_t2o_mou,ml_telco_cdr.loc_ic_t2o_mou,ml_telco_cdr.last_date_of_month,ml_telco_cdr.arpu,ml_telco_cdr.onnet_mou,ml_telco_cdr.offnet_mou,...,ml_telco_cdr.deviceprotection,ml_telco_cdr.techsupport,ml_telco_cdr.streamingtv,ml_telco_cdr.streamingmovies,ml_telco_cdr.contract,ml_telco_cdr.paperlessbilling,ml_telco_cdr.paymentmethod,ml_telco_cdr.monthlyrecharges,ml_telco_cdr.totalcharges,ml_telco_cdr.churn
0,7000000825,303,ASM,0,0,0,30-06-2014 00:00,91.3,0,2.76,...,No,No,Yes,Yes,Month-to-month,Yes,Credit card (automatic),99.3,918.75,No
1,7000000825,303,ASM,0,0,0,31-07-2014 00:00,108.51,2.01,0.41,...,No,No,Yes,No,One year,Yes,Bank transfer (automatic),62.15,2215.45,No
2,7000000825,303,ASM,0,0,0,31-08-2014 00:00,53.11,0,3.85,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),20.65,1057,No
3,7000000825,303,ASM,0,0,0,30-09-2014 00:00,0,,,...,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check,19.95,927.1,No
4,7000000926,303,ASM,0,0,0,30-06-2014 00:00,658.19,85.59,366.78,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),110,7611.85,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39798,7001645485,303,ASM,0,0,0,31-08-2014 00:00,295.66,336.96,282.94,...,No,No,No,No,Month-to-month,No,Electronic check,75.35,273.4,No
39799,7002402763,303,ASM,0,0,0,30-06-2014 00:00,162.77,72.98,117.96,...,No,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),49.85,1520.1,No
39800,7002402763,303,ASM,0,0,0,31-07-2014 00:00,34.45,2.43,15.79,...,Yes,Yes,Yes,Yes,One year,No,Mailed check,113.3,5032.25,No
39801,7002402763,303,ASM,0,0,0,31-08-2014 00:00,90.64,22.88,20.83,...,Yes,Yes,Yes,Yes,Two year,No,Mailed check,88.1,5526.75,No


In [47]:
hive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39803 entries, 0 to 39802
Data columns (total 86 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   ml_telco_cdr.mobile_number           39803 non-null  object
 1   ml_telco_cdr.circle_id               39803 non-null  object
 2   ml_telco_cdr.circle_name             39803 non-null  object
 3   ml_telco_cdr.loc_og_t2o_mou          39803 non-null  object
 4   ml_telco_cdr.std_og_t2o_mou          39803 non-null  object
 5   ml_telco_cdr.loc_ic_t2o_mou          39803 non-null  object
 6   ml_telco_cdr.last_date_of_month      39803 non-null  object
 7   ml_telco_cdr.arpu                    39803 non-null  object
 8   ml_telco_cdr.onnet_mou               39803 non-null  object
 9   ml_telco_cdr.offnet_mou              39803 non-null  object
 10  ml_telco_cdr.roam_ic_mou             39803 non-null  object
 11  ml_telco_cdr.loc_og_t2t_mo           3980

In [48]:
# df['ml_telco_cdr.arpu'] = df['ml_telco_cdr.arpu'].astype('float')
# df['ml_telco_cdr.arpu'].dtype
csv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398295 entries, 0 to 398294
Data columns (total 86 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   MOBILE_NUMBER           398295 non-null  int64  
 1   CIRCLE_ID               398295 non-null  int64  
 2   CIRCLE_NAME             398295 non-null  object 
 3   LOC_OG_T2O_MOU          394895 non-null  float64
 4   STD_OG_T2O_MOU          394895 non-null  float64
 5   LOC_IC_T2O_MOU          394895 non-null  float64
 6   LAST_DATE_OF_MONTH      396636 non-null  object 
 7   ARPU                    398295 non-null  float64
 8   ONNET_MOU               379077 non-null  float64
 9   OFFNET_MOU              379077 non-null  float64
 10  ROAM_IC_MOU             379077 non-null  float64
 11  LOC_OG_T2T_MO           379077 non-null  float64
 12  LOC_OG_T2M_MOU          379077 non-null  float64
 13  LOC_OG_T2F_MOU          379077 non-null  float64
 14  LOC_OG_T2C_MOU      

In [40]:
# csv_df['LOC_OG_T2O_MOU']
# hive_df['ml_telco_cdr.loc_og_t2o_mou']
# hive_df['ml_telco_cdr.loc_og_t2o_mou'] = pd.to_numeric(hive_df['ml_telco_cdr.loc_og_t2o_mou'], errors='coerce')
# df['points'] = pd.to_numeric(df['points'], errors='coerce')

In [41]:
hive_df['ml_telco_cdr.loc_og_t2o_mou']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
39798    0.0
39799    0.0
39800    0.0
39801    0.0
39802    0.0
Name: ml_telco_cdr.loc_og_t2o_mou, Length: 39803, dtype: float64

In [51]:

for ind,col in enumerate(hive_df.columns.values):
    try:
#         hive_df[col] = hive_df[col].astype('float')
        hive_df[col] = pd.to_numeric(hive_df[col], errors='ignore')
#         print(f'{col} is converted into the float')
        l.append(ind)
    except Exception as e:
        print(f'{ind} : {col}')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]


86

In [54]:
hive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39803 entries, 0 to 39802
Data columns (total 86 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ml_telco_cdr.mobile_number           39803 non-null  int64  
 1   ml_telco_cdr.circle_id               39803 non-null  int64  
 2   ml_telco_cdr.circle_name             39803 non-null  object 
 3   ml_telco_cdr.loc_og_t2o_mou          39428 non-null  float64
 4   ml_telco_cdr.std_og_t2o_mou          39428 non-null  float64
 5   ml_telco_cdr.loc_ic_t2o_mou          39428 non-null  float64
 6   ml_telco_cdr.last_date_of_month      39803 non-null  object 
 7   ml_telco_cdr.arpu                    39803 non-null  float64
 8   ml_telco_cdr.onnet_mou               37830 non-null  float64
 9   ml_telco_cdr.offnet_mou              37830 non-null  float64
 10  ml_telco_cdr.roam_ic_mou             37830 non-null  float64
 11  ml_telco_cdr.loc_og_t2t_mo  

In [55]:
hive_df['ml_telco_cdr.seniorcitizen']

0        0
1        0
2        0
3        0
4        0
        ..
39798    1
39799    0
39800    0
39801    0
39802    0
Name: ml_telco_cdr.seniorcitizen, Length: 39803, dtype: int64